# Project - Airline AI Assistant

We will make AI Customer Support assistant for an Airline

In [21]:
# imports
import os
import json
from dotenv import load_dotenv
from openai import OpenAI 
import gradio as gr 

In [22]:
# Initialization
load_dotenv(override=True)

openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not found")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()



OpenAI API Key exists and begins sk-proj-


In [23]:
system_message = "You are a helpful asssistant for an Airline called Flight AI 101. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so. "

In [24]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


## Tools 
Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky.. we're giving it the power to run code on our machine?

In [25]:
# Let's start by making a useful function

ticket_prices = {"bali": "$1600", "paris": "$1200", "tokyo": "$1300", "new york": "$1400", "los angeles": "$1500"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [26]:
get_ticket_price("bali")

Tool get_ticket_price called for bali


'$1600'

In [27]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False,
    }
}

In [29]:
# And this is included in a list of tools:
tools = [{"type": "function", "function": price_function}]

## Getting OpenAI to use our Tool
What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.


In [31]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages) 
        
    return response.choices[0].message.content

In [33]:
# We have to write that function handle_tool_call

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get("destination_city")
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"detination_city": city, "price": price}),
        "tool_call_id": tool_call.id,
    }
    return response, city

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for Tokyo
Tool get_ticket_price called for Thailand
Tool get_ticket_price called for Los Angeles
Tool get_ticket_price called for New York
